Use TF IDF Vectoriser to transform text into vectors

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import pandas as pd
from bibrec.server.Utils import *



books = get_books("../data/BX-Books.csv")
book_data = pd.read_csv("../data/editions_dump.csv")
CHUNK_QUANTITY = 50
tfidf = TfidfVectorizer(stop_words='english')
book_data = pd.DataFrame(book_data).drop_duplicates(subset=["isbn_10"], ignore_index=True)



ModuleNotFoundError: No module named 'bibrec'

Preprocessing data to remove unnessecary spaces and characters

In [ ]:
def book_data_preprocessing(self):
    book_data["book_info"] = book_data["book_info"].astype("string")
    book_data["book_info"] = book_data["book_info"].apply(lambda x: x.lower())
    book_data["book_info"] = book_data["book_info"].apply(lambda x: x.replace("(", ''))
    book_data["book_info"] = book_data["book_info"].apply(lambda x: x.replace(")", ''))
    book_data["book_info"] = book_data["book_info"].apply(lambda x: x.replace(',', ''))
    book_data["book_info"] = book_data["book_info"].apply(lambda x: x.replace('.', ''))
    book_data["book_info"] = book_data["book_info"].apply(lambda x: x.replace("'", ''))
    book_data["book_info"] = book_data["book_info"].apply(lambda x: x.replace("--", ''))
    book_data["book_info"] = book_data["book_info"].apply(lambda x: x.replace("  ", ' '))
    book_data["book_info"] = book_data["book_info"].apply(lambda x: ' '.join(list(set(x.split()))))

Create a Similarity Matrix for all books using cosine similarity

In order to avoid running out of memory while calculating the Matrix we decided to split the data into smaller chunks
- This has the effect that only similarities for books in the same chunk are calculated

In [ ]:
def calculate_similarities(self):
    # data cleaning
    self.book_data.rename(columns={'isbn_10': 'isbn', 'first_sentence.value': 'first_sentence', 'notes.value': 'notes'}, inplace=True)
    self.book_data.isbn = self.book_data.isbn.map(lambda x: str(x).strip()[2:-2])
    self.book_data = self.books.merge(self.book_data, on="isbn", how="inner")
    self.book_data = self.book_data.drop(columns=["Unnamed: 0", "isbn_13", "publishers", "title", "first_sentence.type", "notes.type"])
    self.book_data.subjects = self.book_data.subjects.map(lambda x: str(x).strip()[2:-2])
    self.book_data.genres = self.book_data.genres.map(lambda x: str(x).strip()[2:-2])
    self.book_data = self.book_data.fillna(" ")

    # add book_info column and append relevant data
    self.book_data["book_info"] = self.book_data["book_title"] + " " + self.book_data["subjects"] + " " + self.book_data["genres"]
    self.book_data_preprocessing()

    self.similarities = []
    self.mappings = []
    for i in range(0, self.CHUNK_QUANTITY):
        len_books = self.book_data.shape[0] / self.CHUNK_QUANTITY
        chunk_data = self.book_data[int(i * len_books):int((i + 1) * len_books)]
        chunk_data = chunk_data.reset_index(drop=True)
        similarity_matrix = self.get_similarity_matrix(chunk_data)
        self.similarities.append(similarity_matrix)
        self.mappings.append(pd.Series(chunk_data.index, index=chunk_data["isbn13"]))

def get_similarity_matrix(self, book_data):
    tfidf_matrix = self.tfidf.fit_transform(book_data["book_info"])
    similarity_matrix = linear_kernel(tfidf_matrix, tfidf_matrix)
    return similarity_matrix

All similar items are gathered for the given isbn and sorted in a descending order.
The number of returned items is split at n

In [ ]:
def recommend_tf_idf(self, isbn13, n=15):
    book_indices = []
    for i in range(0, self.CHUNK_QUANTITY):
        if isbn13 in self.mappings[i].index:
            similarity_matrix = self.similarities[i]
            book_index = self.mappings[i][isbn13]

            if isinstance(book_index, pd.Series):
                book_index = book_index[0]
            similarity_score = list(enumerate(similarity_matrix[book_index]))
            # Sort the books based on the similarity scores
            similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
            similarity_score = similarity_score[1:n + 1]

            # Check if all similarities are 0
            if all(i[1] == 0 for i in similarity_score):
                return None

            book_indices.append([i[0] for i in similarity_score])

    if len(book_indices) == 0:
        return None
    return self.book_data[["book_title", "isbn13", "isbn"]].iloc[book_indices[0]]

In [ ]:
calculate_similarities()
recommend_tf_idf("9780002005883")